In [2]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [3]:
#Change the date to the previous day
query = """
SELECT *, "Regular Season" as Category FROM `perceptive-ivy-290216.nba_api_box_misc_player.all_time_reg_season`
UNION ALL
SELECT *, "Playoffs" as Category FROM `perceptive-ivy-290216.nba_api_box_misc_player.all_time_playoff`
"""
project_id = "perceptive-ivy-290216"
df_bq_misc_player = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [22]:
#USE FOR ALL TIME DATA
df_bq3=df_bq_misc_player.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year="2023-24"
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq3=df_bq3.fillna(0)
types="Regular Season"
# types="Playoffs"
df_bq2=df_bq3[df_bq3["Category"]==types]
df_bq2.head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,Season,GAME_ID_1,GAME_DATE,MATCHUP,SEASON_1,Category
4591,0022100551,1.610613e+09,MIN,Minnesota,1629675.0,Naz Reid,Naz,C,0,30:24,6.0,7.0,0.0,20.0,15.0,5.0,6.0,24.0,0.0,0.0,4.0,5.0,2021-22,0022100551,2022-01-02,MIN @ LAL,2021-22,Regular Season
4592,0022100100,1.610613e+09,MEM,Memphis,1629630.0,Ja Morant,Ja,G,0,35:02,2.0,6.0,2.0,22.0,9.0,7.0,9.0,30.0,0.0,2.0,0.0,2.0,2021-22,0022100100,2021-11-01,DEN @ MEM,2021-22,Regular Season
4593,0022100505,1.610613e+09,MEM,Memphis,1629630.0,Ja Morant,Ja,G,0,33:42,0.0,3.0,0.0,20.0,18.0,10.0,4.0,34.0,0.0,1.0,2.0,5.0,2021-22,0022100505,2021-12-27,MEM @ PHX,2021-22,Regular Season
4594,0022100554,1.610613e+09,MEM,Memphis,1629630.0,Ja Morant,Ja,G,0,33:34,4.0,8.0,10.0,24.0,9.0,2.0,12.0,32.0,0.0,2.0,0.0,7.0,2021-22,0022100554,2022-01-03,MEM @ BKN,2021-22,Regular Season
4595,0022101047,1.610613e+09,MEM,Memphis,1629630.0,Ja Morant,Ja,G,0,33:05,7.0,2.0,7.0,22.0,15.0,5.0,8.0,38.0,0.0,2.0,2.0,5.0,2021-22,0022101047,2022-03-18,MEM @ ATL,2021-22,Regular Season


In [5]:
#Add Game Number for trended values plotting
df_bq2["Game Number"] = df_bq2.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1758/3610659743.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2["Game Number"] = df_bq2.groupby("PLAYER_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1758/3610659743.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)


In [6]:
#Get cumulative stat for each player
df_bq2['Total PTS_OFF_TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_OFF_TOV'].cumsum()
df_bq2['Total PTS_2ND_CHANCE'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_2ND_CHANCE'].cumsum()
df_bq2['Total PTS_FB'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_FB'].cumsum()
df_bq2['Total PTS_PAINT'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_PAINT'].cumsum()
df_bq2['Total OPP_PTS_OFF_TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OPP_PTS_OFF_TOV'].cumsum()
df_bq2['Total OPP_PTS_2ND_CHANCE'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OPP_PTS_2ND_CHANCE'].cumsum()
df_bq2['Total OPP_PTS_FB'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OPP_PTS_FB'].cumsum()
df_bq2['Total OPP_PTS_PAINT'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['OPP_PTS_PAINT'].cumsum()
df_bq2['Total BLK'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['BLK'].cumsum()
df_bq2['Total BLKA'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['BLKA'].cumsum()
df_bq2['Total PF'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PF'].cumsum()
df_bq2['Total PFD'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PFD'].cumsum()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1758/4020340597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2['Total PTS_OFF_TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_OFF_TOV'].cumsum()
/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_1758/4020340597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bq2['Total PTS_2ND_CHANCE'] = df_bq2.sort_values("GAME_DATE").groupby(['PLAYER_NAME'])['PTS_2ND_CHANCE'].cumsum()
/var/folders/x_/b65sxrpx6737wt

In [7]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[[
       'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD']].sum()).reset_index()
#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['PLAYER_ID','PLAYER_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["PLAYER_ID","PLAYER_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["PLAYER_NAME","PLAYER_ID"],right_on=["PLAYER_NAME","PLAYER_ID"])
df_totals.head()

,PLAYER_ID,PLAYER_NAME,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,GP
0,2.0,Byron Scott,108.0,39.0,96.0,106.0,472.0,461.0,373.0,1220.0,16.0,9.0,72.0,1.0,82
1,3.0,Grant Long,365.0,327.0,242.0,746.0,2709.0,2268.0,1880.0,5876.0,72.0,97.0,793.0,40.0,413
2,7.0,Dan Schayes,79.0,168.0,11.0,192.0,700.0,563.0,571.0,1538.0,51.0,42.0,279.0,2.0,203
3,9.0,Sedale Threatt,17.0,16.0,8.0,10.0,135.0,123.0,105.0,328.0,3.0,4.0,29.0,0.0,26
4,12.0,Chris King,0.0,4.0,0.0,2.0,22.0,20.0,11.0,34.0,1.0,1.0,8.0,0.0,14


In [8]:
sort_category="PTS_OFF_TOV"
# sort_category=[ 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
#        'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
#        'BLK', 'BLKA', 'PF', 'PFD']

In [9]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,PLAYER_ID,PLAYER_NAME,PTS_OFF_TOV,PTS_2ND_CHANCE,PTS_FB,PTS_PAINT,OPP_PTS_OFF_TOV,OPP_PTS_2ND_CHANCE,OPP_PTS_FB,OPP_PTS_PAINT,BLK,BLKA,PF,PFD,GP,RK_Category
900,2544.0,LeBron James,7351.0,3838.0,7970.0,18940.0,18335.0,14721.0,14876.0,49128.0,1114.0,1116.0,2690.0,8027.0,1538,1
399,977.0,Kobe Bryant,5369.0,3195.0,3945.0,11084.0,15512.0,13973.0,12929.0,40992.0,640.0,1160.0,3353.0,4337.0,1449,2
373,947.0,Allen Iverson,4883.0,1909.0,4721.0,8708.0,12851.0,10825.0,10542.0,31084.0,164.0,1187.0,1777.0,2247.0,1009,3
573,1717.0,Dirk Nowitzki,4651.0,3296.0,3262.0,7008.0,15122.0,13869.0,14674.0,42990.0,1281.0,759.0,3601.0,4486.0,1567,4
574,1718.0,Paul Pierce,4475.0,2313.0,2860.0,7946.0,15427.0,12350.0,11413.0,35610.0,745.0,1141.0,3643.0,3671.0,1449,5
1205,201142.0,Kevin Durant,4395.0,2368.0,4704.0,9326.0,13992.0,11481.0,10941.0,36748.0,1203.0,691.0,2039.0,5592.0,1079,6
902,2546.0,Carmelo Anthony,4308.0,3813.0,3385.0,9224.0,14171.0,12382.0,11948.0,38956.0,644.0,1397.0,3543.0,5054.0,1347,7
1274,201566.0,Russell Westbrook,4244.0,2486.0,5366.0,11264.0,13639.0,11162.0,10720.0,38264.0,371.0,1265.0,2986.0,5424.0,1198,8
1337,201935.0,James Harden,4177.0,2024.0,3929.0,7934.0,13246.0,10625.0,10958.0,35756.0,604.0,1132.0,2704.0,5994.0,1111,9
377,951.0,Ray Allen,4136.0,2273.0,3632.0,6360.0,15705.0,13075.0,12089.0,37832.0,244.0,780.0,2858.0,1551.0,1343,10


In [10]:
#Get Player Names of the Top Category we have selected
names=top.PLAYER_NAME.to_list()

In [11]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

In [12]:
#Get Trended Data Point 

In [13]:
fig = px.scatter(
  df_top, 
  x="Game Number", 
  y="Total {}".format(sort_category),
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Trended Player {} vs. Game Number</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['TEAM_ABBREVIATION',
       'PLAYER_NAME',
       'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD', 'Season', 'GAME_ID_1', 'GAME_DATE',
       'MATCHUP', 'SEASON_1', 'Game Number', 'Total PTS_OFF_TOV',
       'Total PTS_2ND_CHANCE', 'Total PTS_FB', 'Total PTS_PAINT',
       'Total OPP_PTS_OFF_TOV', 'Total OPP_PTS_2ND_CHANCE', 'Total OPP_PTS_FB',
       'Total OPP_PTS_PAINT', 'Total BLK', 'Total BLKA', 'Total PF',
       'Total PFD'], 
  height=800, 
  width=1200,
  )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces(marker={'size': 3})

fig

In [ ]:
fig.write_html("{} All Time Trended Player {} vs. Game Number.html".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(sort_category,current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [14]:
cat1="PTS_FB"
cat2="PTS_PAINT"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {} vs. {}</b>".format(types,cat1,cat2),
  template="plotly_white",
  hover_data=['PLAYER_NAME',
       'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [25]:
fig_scatter.write_html("{} All Time Player {} vs. {}.html".format(types,cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Player {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Bar Chart For Category

In [18]:
fig_bar = px.bar(
  df_totals.sort_values(by=sort_category, ascending=False).head(25),
  x='PLAYER_NAME', 
  y=sort_category,
  color='PLAYER_NAME',
  color_discrete_sequence=px.colors.qualitative.G10,
  title="<b>{} All Time Player {}</b>".format(types,sort_category),
  template="plotly_white",
  hover_data=['PLAYER_NAME',
       'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD'], 
  height=800, 
  width=1200,
  )

fig_bar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_bar.update_traces(marker_color='#0047ab')
fig_bar

In [ ]:
fig_bar.write_html("{} All Time Player {}</b>".format(types,sort_category),full_html=False, include_plotlyjs='cdn')
# fig_bar.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [20]:
sort_categories=[ 'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
       'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
       'BLK', 'BLKA', 'PF', 'PFD']

In [ ]:
#TRENDED

In [244]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_multi = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>{} All Time Trended Player {} vs. Game Number</b>".format(types, i),
    template="plotly_white",
    hover_data=['PLAYER_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi.update_traces(marker={'size': 3})
    fig_multi.write_html("{} All Time Trended Player {} vs. Game Number.html".format(types, i),full_html=False, include_plotlyjs='cdn')
    # fig_multi.write_html("Trended Player {} vs. Game Number For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

In [29]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['PLAYER_NAME'].isin(names)]

    fig_bar_multi = px.bar(
    df_totals.sort_values(by=i, ascending=False).head(25),
    x='PLAYER_NAME', 
    y=i,
    color='PLAYER_NAME',
    color_discrete_sequence=px.colors.qualitative.G10,
    title="<b>{} All Time Player {}</b>".format(types,i),
    template="plotly_white",
    hover_data=['PLAYER_NAME',
        'PTS_OFF_TOV', 'PTS_2ND_CHANCE', 'PTS_FB', 'PTS_PAINT',
        'OPP_PTS_OFF_TOV', 'OPP_PTS_2ND_CHANCE', 'OPP_PTS_FB', 'OPP_PTS_PAINT',
        'BLK', 'BLKA', 'PF', 'PFD'], 
    height=800, 
    width=1200,
    )

    fig_bar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_bar_multi.update_traces(marker_color='#0047ab')
    fig_bar_multi.write_html("{} All Time Player {}.html".format(types,i),full_html=False, include_plotlyjs='cdn')
# fig_bar_multi.write_html("Player {} For The {} {}</b>".format(sort_category, current_year, types),full_html=False, include_plotlyjs='cdn')